## Combining Model

In [1]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [2]:
## idea
#reading the data

## constant
genotype_data = pd.read_csv('data/ANMerge_SNP_Covariates.csv') 

## not constant
proteomics_data = pd.read_csv('data/ANMerge_Proteomics_under_90.csv')

##  constant 
gene_expression_data = pd.read_csv('data/ANMerge_gene_expression_normalized_under_90.csv')

## constant
metabolic_data = pd.read_csv('data/ALZ_Serum_HPOS_MS_QI.csv')

## not constant
data_clinical = pd.read_csv('data/ANMerge_clinical_under_90.csv')

## not constant
mri_data = pd.read_csv('data/ANMerge_MRI_FS5.3_under_90.csv')

test_ids = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/test_ids.csv')
train_ids = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/train_ids.csv')

In [3]:
genotype_data.shape, proteomics_data.shape, gene_expression_data.shape, metabolic_data.shape, data_clinical.shape 

((992, 22), (898, 1025), (691, 5222), (556, 2803), (4448, 45))

In [4]:
metabolic_data['Subject_ID']= metabolic_data['Sample ID'].apply(lambda x:x.split('_')[0])
mri_data['Subject_ID']= mri_data['Key'].apply(lambda x:x.split('_')[0])

In [5]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_csv('data/MRI_data_2.csv')
data['Subject_ID']= data['Subject_ID'].apply(lambda x:x.split('_')[0])
feats = list(data.columns) 
feats.remove('Visit')
feats.remove('Diagnosis')
feats.remove('Subject_ID')
X_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
X_train =  data[~data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
y_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])]['Diagnosis'].reset_index(drop=True)
y_train = data[~data['Subject_ID'].isin(test_ids['Subject_ID'])]['Diagnosis'].reset_index(drop=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2,random_state =123)



params = {
    'boosting_type': 'dart',         # Gradient Boosting Decision Tree
    'objective': 'multiclass',           # Binary classification
    'num_classes':3,
     'metric': 'multi_logloss',
    'num_leaves': 31,                # Maximum number of leaves in one tree
    'learning_rate': 0.05,           # Learning rate of boosting process
    'feature_fraction': 0.9,         # Percentage of features to be used per iteration
    'bagging_fraction': 0.8,         # Percentage of data to be bagged per iteration
    'bagging_freq': 5,               # Frequency for bagging
    'verbose': -1,                    # < 0: Fatal, = 0: Error (Warning), = 1: Info, > 1: Debug,
}

# Creating dataset for LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
test_data = lgb.Dataset(X_test, label=y_test)

# Training the model
num_round = 1000  # Number of boosting rounds
bst = lgb.train(params, train_data, num_round, valid_sets=[val_data],  callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(50)])

# Making predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
# Find the class with the highest probability for each sample
y_pred_class = np.argmax(y_pred, axis=1)
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred_class)
print("Accuracy:", accuracy)

/home/fathinah.izzati/.local/lib/python3.11/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[50]	valid_0's multi_logloss: 0.532745
[100]	valid_0's multi_logloss: 0.468691
[150]	valid_0's multi_logloss: 0.428033
[200]	valid_0's multi_logloss: 0.375052
[250]	valid_0's multi_logloss: 0.344995
[300]	valid_0's multi_logloss: 0.331742
[350]	valid_0's multi_logloss: 0.324184
[400]	valid_0's multi_logloss: 0.312333
[450]	valid_0's multi_logloss: 0.300615
[500]	valid_0's multi_logloss: 0.288251
[550]	valid_0's multi_logloss: 0.284743
[600]	valid_0's multi_logloss: 0.284463
[650]	valid_0's multi_logloss: 0.287931
[700]	valid_0's multi_logloss: 0.290533
[750]	valid_0's multi_logloss: 0.296682
[800]	valid_0's multi_logloss: 0.300547
[850]	valid_0's multi_logloss: 0.299853
[900]	valid_0's multi_logloss: 0.306978
[950]	valid_0's multi_logloss: 0.312891
[1000]	valid_0's multi_logloss: 0.323206
Accuracy: 0.7262569832402235


In [6]:
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.72      0.82      0.77        50
           1       0.77      0.73      0.75        56
           2       0.70      0.66      0.68        73

    accuracy                           0.73       179
   macro avg       0.73      0.74      0.73       179
weighted avg       0.73      0.73      0.73       179



In [7]:
# Making predictions
y_pred = bst.predict(data[feats],num_iteration=bst.best_iteration)
# Find the class with the highest probability for each sample
y_pred_class = np.argmax(y_pred, axis=1)
# Evaluate model performance
accuracy = accuracy_score( data['Diagnosis'], y_pred_class)
print("Accuracy:", accuracy)

Accuracy: 0.9365708622398414


In [8]:
data['pred_mri'] = y_pred_class

In [9]:
mri_data = data.copy()

In [10]:
data = pd.read_csv('data/Clinical_data_0.csv')
data['Final_Diagnosis']=data['Final_Diagnosis'].map({'AD':0,'CTL':1,'MCI':2})
data = data[data['Final_Diagnosis'].notna()].reset_index(drop=True)
feats = list(data.columns) 
feats.remove('Visit')
feats.remove('Final_Diagnosis')
feats.remove('Subject_ID')

def label_encode(data):
    #label encode categorical data
    categorical_columns = data.select_dtypes(include='object').columns
    #drop the Subject_ID column and PCA columns from the categorical columns
    if 'Subject_ID' in categorical_columns:
        categorical_columns = categorical_columns.drop('Subject_ID')
    if 'PC1' in categorical_columns:
        categorical_columns = categorical_columns.drop(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
    #encode categorical data but print the mapping
    for column in categorical_columns:
        print(column)
        data[column], mapping_index = data[column].factorize()
        print(mapping_index)
    return data
label_encode(data)






DEMQOL
Index(['Fair', 'Good', 'Very_good', 'Poor'], dtype='object')


,Subject_ID,Visit,Final_Diagnosis,CDR_SOB,CDR_Total,CERAD_A_Total,CERAD_B_Total,CERAD_C_Total,CERAD_D_Total,CERAD_E_Correct,...,Father_Dementia,Fulltime_Education_Years,Geriatric_Depression,Hachinski,MOCA,Mother_Dementia,NPI,Sensory_Impairment,TICSM,Webster
0,DCR00001,1,0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,...,0.0,17.0,NaN,8.0,NaN,0.0,20.0,1.0,NaN,2.0
1,DCR00001,2,0,15.0,3.0,NaN,NaN,NaN,NaN,NaN,...,0.0,17.0,NaN,8.0,NaN,0.0,87.0,1.0,NaN,22.0
2,DCR00003,1,0,4.0,0.5,NaN,NaN,NaN,NaN,NaN,...,0.0,11.0,NaN,NaN,NaN,0.0,0.0,1.0,NaN,0.0
3,DCR00003,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,11.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,0.0
4,DCR00003,3,0,2.5,0.5,NaN,NaN,NaN,NaN,NaN,...,0.0,11.0,NaN,2.0,NaN,0.0,8.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4443,TLSMCI611,2,2,0.5,0.5,NaN,9.0,NaN,NaN,5.0,...,0.0,8.0,13.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN
4444,TLSMCI612,1,0,0.5,0.5,NaN,13.0,NaN,NaN,0.0,...,0.0,8.0,0.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN
4445,TLSMCI612,2,0,1.5,0.5,NaN,11.0,NaN,NaN,2.0,...,0.0,8.0,0.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN
4446,TLSMCI613,1,2,0.5,0.5,NaN,14.0,NaN,NaN,6.0,...,0.0,6.0,2.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN


In [11]:

X_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
X_train =  data[~data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
y_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])]['Final_Diagnosis'].reset_index(drop=True)
y_train = data[~data['Subject_ID'].isin(test_ids['Subject_ID'])]['Final_Diagnosis'].reset_index(drop=True)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2,random_state =123)



from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [12]:
# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report                       
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8711864406779661
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       248
           1       0.85      0.94      0.89       217
           2       0.72      0.70      0.71       125

    accuracy                           0.87       590
   macro avg       0.85      0.85      0.85       590
weighted avg       0.88      0.87      0.87       590



In [13]:
# Make predictions on the testing data
y_pred = rf_classifier.predict(data[feats])

# Evaluate the model's performance
accuracy = accuracy_score(data['Final_Diagnosis'], y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9321043165467626


In [14]:
data['pred_clin'] = y_pred

In [15]:
clinical_data = data.copy()

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

data = pd.read_csv('data/Metabolic_data_2.csv')
data['Subject_ID']= data['Sample ID'].apply(lambda x:x.split('_')[0])
feats = list(data.columns) 
feats.remove('Sample ID')
feats.remove('Subject_ID')
feats.remove('Diagnosis')

X_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
X_train =  data[~data['Subject_ID'].isin(test_ids['Subject_ID'])][feats].reset_index(drop=True)
y_test = data[data['Subject_ID'].isin(test_ids['Subject_ID'])]['Diagnosis'].reset_index(drop=True)
y_train = data[~data['Subject_ID'].isin(test_ids['Subject_ID'])]['Diagnosis'].reset_index(drop=True)


# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=len(data['Diagnosis'].unique()), random_state=42)

# Train the classifier on the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.63
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        23
           1       0.62      0.45      0.53        33
           2       0.59      0.75      0.66        44

    accuracy                           0.63       100
   macro avg       0.65      0.62      0.63       100
weighted avg       0.64      0.63      0.62       100



In [17]:

# Make predictions on the testing data
y_pred = xgb_classifier.predict(data[feats])

# Evaluate the model's performance
accuracy = accuracy_score(data['Diagnosis'], y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9332129963898917


In [18]:
data['pred_meta'] = y_pred

In [19]:
metabolic_data = data.copy()

## Eval Salma's Models

In [20]:
folder = "/l/users/salma.hassan/ML703 Project/models/"

In [21]:
import pandas as pd
genotype_dfc = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/genotype.csv')
preteo_dfc = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/final_proteomics.csv')
geneexp_dfc = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/final_gene_expression.csv')

In [22]:
import pickle
import joblib  # You might need to install joblib if not available
## CTL, AD, MCI
# Load the RandomForest model
with open(folder+'Genotype_random_forest_0.422.pkl', 'rb') as file:
    rf_model_feats = pickle.load(file)

rf_model = joblib.load(folder + 'Genotype_random_forest_0.422.pkl')
best_model = rf_model.best_estimator_
genotype_dfc2 = genotype_dfc[rf_model_feats]
genotype_dfc['pred_gen']=best_model.predict(genotype_dfc2)


In [23]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(genotype_dfc['Diagnosis'].map({'AD':1, 'CTL':0, 'MCI':2}), genotype_dfc['pred_gen'])

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.7076612903225806


In [24]:
test_dfc = genotype_dfc[genotype_dfc['Subject_ID'].isin(test_ids['Subject_ID'])]
accuracy_score(test_dfc['Diagnosis'].map({'AD':1, 'CTL':0, 'MCI':2}), test_dfc['pred_gen'])

0.4221105527638191

In [25]:
## AD. CTL. MCI.
lgbm_proteomics_model = joblib.load(folder + 'lgbm_model_proteomics_best_0.856.pkl')
preteo_dfc['pred_prot'] = lgbm_proteomics_model.predict(preteo_dfc.drop(columns='Diagnosis'))

/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [26]:
preteo_dfc['Subject_ID'] = proteomics_data['Unnamed: 0'].apply(lambda x: x.split('_')[0])
preteo_dfc['Visit'] = proteomics_data['Visit'].astype(int)

In [27]:
accuracy_score(preteo_dfc['Diagnosis'], preteo_dfc['pred_prot'])

0.9710467706013363

In [28]:
preteo_dfc.shape

(898, 457)

In [29]:
test_dfc = preteo_dfc[preteo_dfc['Subject_ID'].isin(test_ids['Subject_ID'])]
print(test_dfc.shape)
accuracy_score(test_dfc['Diagnosis'], test_dfc['pred_prot'])

(122, 457)


0.9672131147540983

In [30]:
## AD. MCI. CTL
lgbm_geneexp_model = joblib.load(folder + 'lgbm_model_gene_Expression_0.683.pkl')
geneexp_dfc['pred_gx'] = lgbm_geneexp_model.predict(geneexp_dfc.drop(columns='Diagnosis'))
geneexp_dfc['Subject_ID'] = gene_expression_data['Unnamed: 0'].apply(lambda x: x.split('_')[0])

/home/fathinah.izzati/miniconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [31]:
accuracy_score(geneexp_dfc['Diagnosis'], geneexp_dfc['pred_gx'])

0.9363241678726484

In [32]:
test_dfc = geneexp_dfc[geneexp_dfc['Subject_ID'].isin(test_ids['Subject_ID'])]
accuracy_score(test_dfc['Diagnosis'], test_dfc['pred_gx'])

0.9344262295081968

## Merge All

In [33]:
geneexp_dfc.shape, preteo_dfc.shape, genotype_dfc.shape, clinical_data.shape, metabolic_data.shape, mri_data.shape

((691, 195), (898, 457), (992, 14), (4448, 26), (554, 1381), (1009, 129))

In [34]:
id = 'Subject_ID'
geneexp_dfc[id].nunique(), preteo_dfc[id].nunique(), genotype_dfc[id].nunique(), clinical_data[id].nunique(), metabolic_data[id].nunique(), mri_data[id].nunique()

(691, 657, 992, 1695, 554, 365)

In [35]:
geneexp_dfc.rename(columns={"Diagnosis": "Diagnosis_gx"}, inplace=True)
geneexp_dfc.head()

,MMSE,Zuwq1J40IOkIA0SX_w,Z3eg_6C6_giwWUKUU0,QjIAoqJx3SnkJpBLlQ,EJJIm5SWS7_ol4KCHg,NK57uluO5IiS4qKvTg,3oUnEA30oUnSDgDlqk,6muiWUUA5I.24AV8ao,00buPhOysV5Rpc5VF4,lxIFNR_FwcXVelmd2c,...,BwuX4noOqCqhCrS3to,Tovv9pKpft3R5P60qo,E7rpO5ew1Scd6CeKyo,Z5IIAIiUVcOoVci554,co7ogKQj4IRKKS6adc,9g9NAMC57aepuunRlw,95JQ5IA6tJ1.iBV1UU,Diagnosis_gx,pred_gx,Subject_ID
0,0.7,0.606887,0.301105,0.663875,0.302871,0.217396,0.365636,0.375534,0.309962,0.189628,...,0.481636,0.363840,0.614467,0.426161,0.658222,0.474483,0.405516,0,0,DCR00001
1,0.9,0.628548,0.365279,0.763313,0.244604,0.320397,0.218306,0.485195,0.238711,0.171676,...,0.573980,0.359880,0.608641,0.420296,0.445814,0.434047,0.297593,1,1,DCR00004
2,1.0,0.720705,0.542325,0.734361,0.292866,0.330565,0.856146,0.686165,0.274571,0.141723,...,0.485329,0.327550,0.852822,0.263564,0.556588,0.602272,0.412524,2,2,DCR00009
3,1.0,0.679354,0.448841,0.637443,0.455099,0.221630,0.494124,0.434009,0.486266,0.289252,...,0.304816,0.348586,0.639070,0.324178,0.527380,0.532923,0.268632,2,2,DCR00013
4,1.0,0.465901,0.411512,0.430314,0.420020,0.524417,0.158394,0.613181,0.355490,0.165683,...,0.335874,0.302250,0.495087,0.214806,0.488757,0.303398,0.302386,2,2,DCR00014


In [36]:
preteo_dfc.rename(columns={"Diagnosis": "Diagnosis_prot"}, inplace=True)
preteo_dfc.head()

,P01008,Q9UHL4,Q06323,P03952,Q9BWP8,Q9UJ71,P35475,Q9H098,P05089,P49763,...,O60486,P14091,P24071,"P0C0L4,P0C0L5",P04818,O60603,Diagnosis_prot,pred_prot,Subject_ID,Visit
0,35361.7,5829.3,5945.8,50989.4,11320.4,628.2,3813.5,2113.0,579.1,992.9,...,2366.3,701.7,319.6,157940.0,714.4,1356.3,0,0,DCR00001,1
1,38632.5,3607.7,5490.7,24565.5,11148.0,607.5,2518.3,2242.6,538.6,855.8,...,2615.6,758.3,311.6,171192.2,762.2,1985.4,0,0,DCR00007,1
2,36681.9,4612.7,8148.4,34041.7,7432.5,648.1,3109.2,1963.0,603.7,1008.2,...,2565.9,755.7,1559.9,157261.4,825.6,1530.3,0,0,DCR00023,1
3,35014.8,4868.7,6009.0,42033.9,7578.9,630.4,4053.7,2136.6,601.2,903.3,...,2329.4,1224.2,295.0,190670.9,6286.7,1294.3,0,0,DCR00024,1
4,41250.4,4171.4,5444.3,37370.1,6232.1,613.6,3103.4,1720.8,542.2,875.9,...,2268.0,985.3,545.4,164329.4,729.2,1121.9,0,0,DCR00025,1


In [37]:
genotype_dfc.rename(columns={"Diagnosis": "Diagnosis_gx"}, inplace=True)
genotype_dfc.head()

,Subject_ID,Sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Diagnosis_gx,pred_gen
0,DCR00003,2.0,0.0107,0.0286,0.0197,-0.0194,0.0059,26.0000,-0.0087,-0.0331,-0.0026,-0.0203,AD,0
1,DCR00004,1.0,0.0109,0.0258,0.0167,-0.0104,0.0069,-0.0061,-0.0168,-0.0043,0.0151,23.0000,MCI,2
2,DCR00005,2.0,0.0131,0.0225,0.0117,-0.0087,-0.0085,-24.0000,5.0000,0.0007,0.0178,-0.0119,MCI,2
3,DCR00006,2.0,0.0084,0.0279,0.0137,-0.0148,-0.0096,-0.0036,0.0126,0.0123,-0.0017,-0.0117,MCI,1
4,DCR00009,2.0,0.0088,0.0275,0.0137,-0.0178,-0.0104,-0.0052,-0.0082,-0.0286,16.0000,-0.0169,CTL,0


In [38]:
clinical_data.rename(columns={"Final_Diagnosis": "Diagnosis_clin"}, inplace=True)
clinical_data.head()

,Subject_ID,Visit,Diagnosis_clin,CDR_SOB,CDR_Total,CERAD_A_Total,CERAD_B_Total,CERAD_C_Total,CERAD_D_Total,CERAD_E_Correct,...,Fulltime_Education_Years,Geriatric_Depression,Hachinski,MOCA,Mother_Dementia,NPI,Sensory_Impairment,TICSM,Webster,pred_clin
0,DCR00001,1,0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,...,17.0,NaN,8.0,NaN,0.0,20.0,1.0,NaN,2.0,0
1,DCR00001,2,0,15.0,3.0,NaN,NaN,NaN,NaN,NaN,...,17.0,NaN,8.0,NaN,0.0,87.0,1.0,NaN,22.0,0
2,DCR00003,1,0,4.0,0.5,NaN,NaN,NaN,NaN,NaN,...,11.0,NaN,NaN,NaN,0.0,0.0,1.0,NaN,0.0,0
3,DCR00003,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,0.0,0
4,DCR00003,3,0,2.5,0.5,NaN,NaN,NaN,NaN,NaN,...,11.0,NaN,2.0,NaN,0.0,8.0,1.0,NaN,0.0,0


In [39]:
metabolic_data.rename(columns={"Diagnosis": "Diagnosis_meta"}, inplace=True)
metabolic_data.head()

,2.68_380.3754m/z,2.69_915.9888m/z,2.67_379.2146n,2.70_952.9981m/z,2.70_1049.5313m/z,2.69_325.2245n,2.70_263.1581m/z,2.70_250.1790m/z,2.65_561.8540m/z,2.65_280.1997n,...,0.65_334.1403m/z,0.87_197.1651m/z,0.62_431.1429m/z,1.02_111.0548m/z,0.64_439.2432m/z,0.69_228.2312m/z,Sample ID,Diagnosis_meta,Subject_ID,pred_meta
0,0.0,0.141887,0.155377,0.475351,0.320422,0.001610,0.000283,0.000481,0.163862,0.024318,...,0.025333,0.117418,0.033534,0.527260,0.016686,0.069577,LDZADC003_1,0,LDZADC003,0
1,0.0,0.841281,0.182323,0.644411,0.520985,0.002147,0.000354,0.000677,0.192817,0.025249,...,0.096711,0.054526,0.037839,0.134347,0.024781,0.058054,LNDCTL019_1,1,LNDCTL019,1
2,0.0,0.011674,0.118177,0.454199,0.295365,0.001754,0.000376,0.000548,0.281775,0.030040,...,0.023422,0.048747,0.015370,0.182499,0.018988,0.077321,THSADC057_1,0,THSADC057,0
3,0.0,0.000243,0.000055,0.000068,0.000000,0.001085,0.000369,0.000527,0.000000,0.000126,...,0.040493,0.036124,0.015749,0.115501,0.019358,0.063863,PRGMCI017_1,2,PRGMCI017,2
4,0.0,0.000803,0.000054,0.000000,0.000000,0.000528,0.000271,0.000555,0.000000,0.000041,...,0.170218,0.047522,0.027212,0.271051,0.027808,0.152196,KPOMCI041_1,2,KPOMCI041,2


In [40]:
mri_data.rename(columns={"Diagnosis": "Diagnosis_mri"}, inplace=True)
mri_data.head()

,Visit,Site,Sex,Age,APOE,MMSE,lh_bankssts_thickness,rh_bankssts_thickness,lh_caudalanteriorcingulate_thickness,rh_caudalanteriorcingulate_thickness,...,lhCerebralWhiteMatterVol,SubCortGrayVol,MaskVol,BrainSegVol-to-eTIV,MaskVol-to-eTIV,lhSurfaceHoles,rhSurfaceHoles,Subject_ID,Diagnosis_mri,pred_mri
0,0,0,0,86.0,0,20.0,2.224,2.249,2.883,2.172,...,184191.9575,40324.0,1361906.0,0.664587,0.977432,24.0,13.0,KPOADC001,0,0
1,1,0,0,86.0,0,16.0,2.140,2.092,2.991,2.038,...,192981.9801,38797.0,1319352.0,0.659338,0.941138,23.0,17.0,KPOADC001,0,0
2,2,0,0,87.0,0,21.0,2.248,2.343,3.088,2.157,...,189605.4005,39722.0,1342921.0,0.660968,0.959201,30.0,24.0,KPOADC001,0,0
3,0,0,0,68.0,0,26.0,1.971,2.018,2.394,2.336,...,199432.8506,45318.0,1515148.0,0.652202,0.976865,22.0,21.0,KPOADC002,0,0
4,1,0,0,68.0,0,25.0,2.066,2.112,2.353,2.285,...,195086.1362,44336.0,1508917.0,0.643059,0.968317,20.0,9.0,KPOADC002,0,0


In [41]:
clinical_data = clinical_data[clinical_data['Visit']==1].reset_index(drop=True)

In [42]:
preteo_dfc = preteo_dfc[preteo_dfc['Visit']==1].reset_index(drop=True)

In [43]:
mri_data = mri_data[mri_data['Visit']==0].reset_index(drop=True)

In [44]:
# ## Join all
# merged_df = pd.merge(preteo_dfc,mri_data, how='left', on=['Subject_ID','Visit'])
# print(merged_df.shape)
# # merged_df = pd.merge(merged_df, genotype_dfc, how='left', on=['Subject_ID'])
# # print(merged_df.shape)
# merged_df = pd.merge(merged_df,clinical_data , how='left', on=['Subject_ID','Visit'])
# print(merged_df.shape)
# merged_df = pd.merge(merged_df, metabolic_data, how='left', on=['Subject_ID'])
# print(merged_df.shape)
# merged_df = pd.merge(merged_df, geneexp_dfc , how='left', on=['Subject_ID'])
# print(merged_df.shape)

In [45]:
## Join all
merged_df = pd.merge(geneexp_dfc, metabolic_data , how='left', on=['Subject_ID'])
print(merged_df.shape)
# merged_df = pd.merge(merged_df, genotype_dfc, how='left', on=['Subject_ID'])
# print(merged_df.shape)
merged_df = pd.merge(merged_df, preteo_dfc, how='left', on=['Subject_ID'])
print(merged_df.shape)
merged_df = pd.merge(merged_df, mri_data, how='left', on=['Subject_ID'])
print(merged_df.shape)
merged_df = pd.merge(merged_df, clinical_data , how='left', on=['Subject_ID'])
print(merged_df.shape)

(691, 1575)
(691, 2031)
(713, 2159)
(713, 2184)


In [46]:
for i in merged_df.columns:
    if i[:4]=='pred':
        print('"'+i+'"', end=', ')

"pred_gx", "pred_meta", "pred_prot", "pred_mri", "pred_clin", 

In [47]:
merged_df['pred_gx'] = merged_df['pred_gx'].map({0:0, 2:1, 1:2})
# merged_df['pred_gen'] = merged_df['pred_gen'].map({0:1, 1:0, 2:2})

## Test Data Evaluation

In [48]:
# import pandas as pd

# test_ids = pd.read_csv('/l/users/salma.hassan/ML703 Project/inputs/test_ids.csv')
# merged_df = pd.read_csv('Merged_DF.csv')

In [52]:
pred_df = merged_df[[ "pred_mri", "pred_prot", "pred_gx","pred_meta","pred_clin"]]

## pake mri gapake sama aja

In [53]:
pred_df.isna().sum(axis=0)/4469*100

pred_mri     8.614903
pred_prot    8.055493
pred_gx      0.000000
pred_meta    6.466771
pred_clin    0.000000
dtype: float64

In [54]:
# Define a custom function to determine the majority vote with a specific rule for ties
def custom_majority_vote(row):
    # Calculate the mode ignoring NaNs
    mode_vals = row.mode(dropna=True)
    # If there's a tie or no clear mode, use 'pred_clin' as the default
    if len(mode_vals) == 1:
        return mode_vals.iloc[0]
    elif not np.isnan(row['pred_prot']):
        return row['pred_prot']
    elif not np.isnan(row['pred_gx']):
        return row['pred_gx']
    elif not np.isnan(row['pred_meta']):
        return row['pred_meta']
    elif not np.isnan(row['pred_mri']):
        return row['pred_mri']
    elif not np.isnan(row['pred_clin']):
        return row['pred_clin']

# Apply the function across the DataFrame rows
pred_df['Majority_Vote'] = pred_df.apply(custom_majority_vote, axis=1)
# pred_df['Majority_Vote'] = pred_df.mode(axis=1, numeric_only=True, dropna=True)[0]  # [0] to select the first mode in case of ties


/tmp/slurm-fathinah.izzati-29894/ipykernel_100469/2682051839.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['Majority_Vote'] = pred_df.apply(custom_majority_vote, axis=1)


In [55]:
pred_df['Subject_ID'] = merged_df['Subject_ID']
pred_df['Visit'] = merged_df['Visit']
pred_df['Diagnosis']= merged_df['Diagnosis_gx'].map({0:0, 2:1, 1:2})
# pred_df['Diagnosis']= merged_df['Diagnosis_prot']

/tmp/slurm-fathinah.izzati-29894/ipykernel_100469/3626710396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['Subject_ID'] = merged_df['Subject_ID']
/tmp/slurm-fathinah.izzati-29894/ipykernel_100469/3626710396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['Visit'] = merged_df['Visit']
/tmp/slurm-fathinah.izzati-29894/ipykernel_100469/3626710396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

## All Data Evaluatiom

In [56]:
test_pred_df = pred_df[pred_df['Subject_ID'].isin(test_ids['Subject_ID'])]
train_pred_df = pred_df[~pred_df['Subject_ID'].isin(test_ids['Subject_ID'])]

In [57]:
accuracy_score(train_pred_df['Majority_Vote'], train_pred_df['Diagnosis'])

0.9829351535836177

In [58]:
accuracy_score(test_pred_df['Majority_Vote'], test_pred_df['Diagnosis'])

0.984251968503937

In [62]:
print(classification_report(test_pred_df['Majority_Vote'], test_pred_df['Diagnosis']))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99        38
         1.0       1.00      0.96      0.98        48
         2.0       0.98      1.00      0.99        41

    accuracy                           0.98       127
   macro avg       0.98      0.99      0.98       127
weighted avg       0.98      0.98      0.98       127



In [60]:
xx = test_pred_df[test_pred_df['Majority_Vote'].notna()]
accuracy_score(xx['Majority_Vote'], xx['Diagnosis'])

0.984251968503937